## Individual Vertebrae Segmentation Process Flow

In [ ]:
from utils.utils import loadMatData
from utils.imgproc import findCentroidSliceIdx, cdfFromHist, findClosestValueIdx, getLocalStack, addStack
from scipy.io import loadmat
from scipy.stats import relfreq
import numpy as np
from matplotlib import pyplot as plt
import cv2

### Load some test data.

In [ ]:
vol_dict = loadmat("C:/.py_workspace/reveal/unet2D/.predictions/pred_vol_p2d3.mat")
pt_vol = vol_dict['pt']
mask_vol = vol_dict['spine']

### Find centroid slice along saggital axis of the spinal mask volume.
#### (use this as a reference location for futhrer processing)

In [ ]:
idx = findCentroidSliceIdx(mask_vol)

### Aggregate PET and mask slices around the centroid slice.
#### (smooths noise)

In [ ]:
# stack and aggregate saggital PET scans around center
pt_stack = getLocalStack(pt_vol, idx, span = 5, dim = 1)
pt_aggregate = addStack(pt_stack, dim = 1)

# stack and aggregate mask slices around center
mask_stack = getLocalStack(mask_vol, idx, span = 5, dim = 1)
mask_aggregate = addStack(mask_stack, dim = 1).astype('bool').astype('int')

# plt.imshow(mask_aggregate, interpolation='none', cmap = 'gray')

### Use morphological filters to "regularize" the aggregate mask.
#### (neccessary as intermediate step)

In [ ]:
# morphological filters to regulularize column shape
mask_aggregate = mask_aggregate.astype('uint8')
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, ksize = (2, 2))
# fill small gaps with CLOSE filter
mask_aggregate = cv2.morphologyEx(mask_aggregate, cv2.MORPH_CLOSE, kernel)
# remove penninsulas with OPEN filter
mask_aggregate = cv2.morphologyEx(mask_aggregate, cv2.MORPH_OPEN, kernel)

# plt.imshow(mask_aggregate, interpolation='none', cmap = 'gray')

### Mask the PET scan with the filtered aggregate mask image.

In [ ]:
pt_aggregate = np.multiply(pt_aggregate, mask_aggregate.astype('int'))

plt.imshow(pt_aggregate, interpolation='none', cmap = 'gray')

### Construct an array contaning the mean of the three central non-zero values in the coronal direction for each axial slice.
#### (this destorys the image)

In [ ]:
ncols = pt_aggregate.shape[1]
pt_data_nonzero = np.empty(ncols, dtype = 'object')
for col in range(ncols):
     col_data = pt_aggregate[:, col]
     col_data_nonzero = col_data[col_data != 0]
     pt_data_nonzero[col] = col_data_nonzero

pt_data_mean = np.empty(ncols, dtype = 'object')
for idx, array in enumerate(pt_data_nonzero):
     if len(array) < 3:
          pt_data_mean[idx] = 0
     else:
          center_idx = int(np.floor(len(array)/2))
          center_vals = [array[center_idx-1], array[center_idx], array[center_idx+1]]
          pt_data_mean[idx] = np.mean(center_vals)

### Algorithm to find "boundary" indexes between vertebrae

In [ ]:
pt_mean_ddx = np.gradient(pt_data_mean)
vspan = 7
spacing = 2

In [ ]:
# find first nonzero index
for idx, val in enumerate(pt_data_mean):
    if val == 0:
        continue
    else:
        start_idx = idx
        break

In [ ]:
# detect dips in first expected span
current_idx = start_idx
vert_idxs = np.zeros(19)
for array_idx in range(19):
    sub_idx = np.argmin(pt_data_mean[current_idx+spacing:current_idx+vspan]) + spacing
    current_idx = current_idx + sub_idx
    vert_idxs[array_idx] = current_idx
    current_idx = current_idx + 1